In [1]:
# export
import Financial_Indicators as fi
import numpy as np
import pandas as pd
import pickle

In [2]:
# export
def load_data():
    file = open("df_all.pkl","rb")
    df_all = pickle.load(file)
    return df_all

In [3]:
# export
def f_temp(x, thr):
    if x >= thr:
        return 1
    elif x <= 1- thr:
        return -1
    else:
        return 0


def f_temp3(x, thr):
    idx = np.argmax(x)
    if (idx == 0) & (x[idx] >= thr):
        return -1
    elif (idx == 1) & (x[idx] >= thr):
        return 0
    elif (idx == 2) & (x[idx] >= thr):
        return 1
    else:
        return -99    
    
    
def f_classif_report(clf, X, y, thr = 0.5):
    orig_len = len(y)
    
    if len(clf.predict_proba(X)[0]) == 3:
        y_pred = [f_temp3(x, thr) for x in clf.predict_proba(X)]
        np_array = np.array(y_pred)
        item_index = np.where((np_array==-1) | (np_array==1) | (np_array==0) )
        item_index = list(item_index[0])
        y_pred = [y_pred[x] for x  in item_index ]
        y_list = [x[0] for x in  y.values]
        y = [y_list[x] for x  in item_index ]
    elif len(clf.predict_proba(X)[0]) == 2:
        y_pred = [f_temp(x[1], thr) for x in clf.predict_proba(X)]
        np_array = np.array(y_pred)
        item_index = np.where((np_array==-1) | (np_array==1) )
        item_index = list(item_index[0])
        y_pred = [y_pred[x] for x  in item_index ]
        y_list = [x[0] for x in  y.values]
        y = [y_list[x] for x  in item_index ]
    
    last_len = len(y)
    
    loss = 1 - (last_len/orig_len)
    print("Lost on dataset: {x}".format(x = loss ) )  
    print(classification_report(y, y_pred))
    print(confusion_matrix(y, y_pred))
    return y, y_pred, loss


def f_prep_df_to_ML(df, is_train = True):
    df_tmp = df.copy()
    if is_train:
        df_tmp.dropna(inplace = True)
    y = df_tmp.loc[:, ["Target"]]
    X = df_tmp.loc[:, [ x for x in df_tmp.columns if x not in (["Target" 
                                                                ,"coin" 
                                                                ,"next1price" 
                                                                ,"next2price" 
                                                                ,"next3price" 
                                                                ,"price"
                                                               # ,"is_day_end"
                                                               # ,"last_time"
                                                               ])  ]]

    return X, y 


In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")
from matplotlib import pyplot as plt
from sklearn.multiclass import OneVsRestClassifier
#from sklearn.svm import SVC

df_all_coins = load_data()

model_dict = {}
list_all = []
#for coin in df_all_coins.coin.unique():
for coin in ['DOGEUSDT']: #DOGEUSDT, LUNAUSDT, XRPUSDT,UNIUSDT, FILUSDT
    df_tmp = df_all_coins.loc[df_all_coins.coin==coin, : ]
    
    #df_tmp = df_all_coins.loc[(df_all_coins.coin==coin) & (df_all_coins.Target != 0), : ]
    X, y = f_prep_df_to_ML(df_tmp)
    
    clf = RandomForestClassifier(n_estimators=10 ,random_state=0, max_depth = 5, class_weight = "balanced")
    #clf.fit(X , y)
    clf = OneVsRestClassifier(clf).fit(X, y)

    del df_tmp
    
    thr = 0.4
    is_model_valid = False
    while thr < 0.91:
        try:
            y_true, y_pr, loss = f_classif_report(clf, X, y, thr)
            f1 = f1_score(y_true, y_pr, average='macro')
            print("{coin} model trial, f1_Score: {f1_Score} and loss is: {loss}".format(coin =coin ,f1_Score = str(f1), loss = str(loss) ))
            if (f1 > 0.8) & (loss <= 0.7):
                is_model_valid = True
                print("model is OK")
                break;
            #thr = thr + 0.02       
        except Exception as e:
            print(str(e))
            pass
        thr = thr + 0.02
    
    if is_model_valid:
        model_dict.update ({coin: [clf, thr]})
    #graph for feature importance

    #from sklearn.metrics import f1_score
    #f1_score(y, y_pred, average='macro')    
    print('**********************')

pickle_out = open("model_dict.pkl","wb")
pickle.dump(model_dict, pickle_out)
pickle_out.close()    

Lost on dataset: 0.06710158434296365
              precision    recall  f1-score   support

          -1       0.67      0.71      0.69       399
           0       0.85      0.86      0.85      1137
           1       0.72      0.66      0.69       466

    accuracy                           0.78      2002
   macro avg       0.75      0.74      0.74      2002
weighted avg       0.78      0.78      0.78      2002

[[283  76  40]
 [ 81 977  79]
 [ 61  97 308]]
DOGEUSDT model trial, f1_Score: 0.7436990791636169 and loss is: 0.06710158434296365
Lost on dataset: 0.11136999068033548
              precision    recall  f1-score   support

          -1       0.70      0.72      0.71       381
           0       0.86      0.88      0.87      1091
           1       0.74      0.67      0.70       435

    accuracy                           0.80      1907
   macro avg       0.76      0.76      0.76      1907
weighted avg       0.80      0.80      0.80      1907

[[274  70  37]
 [ 66 958  67]
 [ 5

In [77]:
X_c = X.copy()

X.loc[:,"proba_1"] = [x[0] for x in clf.predict_proba(X_c)] 
X.loc[:,"proba_0"] = [x[1] for x in clf.predict_proba(X_c)] 
X.loc[:,"proba1"] = [x[2] for x in clf.predict_proba(X_c)] 
X


,vol,tradefreq,last_time,is_day_end,rsi14,rsi28,rsi42,rsi48,rsi72,macd12,signal12,hist12,macd6,signal6,hist6,macd12_ratio,signal12_ratio,proba_1,proba_0,proba1
dt,,,,,,,,,,,,,,,,,,,,
2020-10-27 16:00:00,45873.6100,6411.0,0,0,55.254615,48.234309,43.667941,42.375068,40.993625,1.510030,1.598980,-0.088950,0.409555,0.692531,-0.282976,0.043367,0.045922,0.601366,0.278533,0.120101
2020-10-27 20:00:00,78667.0400,10851.0,0,0,56.061620,48.628672,43.930381,42.606430,41.147945,1.440128,1.567209,-0.127081,0.383433,0.568892,-0.185459,0.040948,0.044562,0.642128,0.276630,0.081242
2020-10-28 00:00:00,96548.6000,14263.0,0,1,54.049344,47.903238,43.532150,42.274166,40.939819,1.318459,1.517459,-0.199000,0.262347,0.446274,-0.183927,0.038218,0.043986,0.629541,0.290809,0.079650
2020-10-28 04:00:00,86359.9200,10470.0,0,0,51.957380,47.143573,43.115122,41.926358,40.722255,1.155928,1.445153,-0.289225,0.085508,0.301968,-0.216459,0.034199,0.042756,0.540733,0.309946,0.149321
2020-10-28 08:00:00,78326.5500,8598.0,0,0,49.069466,46.073706,42.525874,41.434776,40.414732,0.942336,1.344590,-0.402254,-0.155205,0.119098,-0.274304,0.028718,0.040977,0.451574,0.438367,0.110058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-07 20:00:00,33687.0712,11650.0,0,0,42.987091,46.628092,47.881582,48.208699,49.257931,-0.614128,-0.502423,-0.111704,-0.532539,-0.297935,-0.234604,-0.004089,-0.003345,0.164687,0.592489,0.242823
2021-05-07 22:00:00,74622.9410,22144.0,0,0,43.337387,46.766184,47.962521,48.276400,49.297297,-0.718776,-0.545694,-0.173082,-0.654988,-0.440756,-0.214232,-0.004781,-0.003630,0.236406,0.481662,0.281932
2021-05-08 00:00:00,78007.7876,17344.0,0,1,46.599010,48.052808,48.715625,48.906039,49.662851,-0.702778,-0.577111,-0.125667,-0.541964,-0.481239,-0.060725,-0.004638,-0.003809,0.141892,0.633876,0.224232


In [75]:
model_dict.get("DOGEUSDT")

[OneVsRestClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                      max_depth=5,
                                                      n_estimators=10,
                                                      random_state=0)),
 0.4800000000000001]

In [25]:
model_dict

{'DOGEUSDT': [OneVsRestClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                       max_depth=5,
                                                       n_estimators=10,
                                                       random_state=0)),
  0.4600000000000001]}

In [76]:
from exporter import export
export("MLTrain.ipynb","MLTrain.py")